In [17]:
from google.cloud import storage

In [18]:
storage_client = storage.Client()

In [ ]:
bucket_name = "10bt_gpt4"

In [26]:
bucket = storage_client.bucket(bucket_name)
print(f"bucket {bucket.name} accessed.")

bucket 10bt_gpt4 accessed.


In [27]:
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0

    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )


In [28]:
upload_blob("10bt_gpt4", "/Users/chinmay/Programming/deep_learning/data/edu_fineweb10B/edufineweb_train_000001.npy", "edufineweb_train_000001.npy")

File /Users/chinmay/Programming/deep_learning/data/edu_fineweb10B/edufineweb_train_000001.npy uploaded to edufineweb_train_000001.npy.


In [37]:
def upload_many_blobs_with_transfer_manager(
    bucket_name, filenames, source_directory="", workers=8
):
    """Upload every file in a list to a bucket, concurrently in a process pool.

    Each blob name is derived from the filename, not including the
    `source_directory` parameter. For complete control of the blob name for each
    file (and other aspects of individual blob metadata), use
    transfer_manager.upload_many() instead.
    """

    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # A list (or other iterable) of filenames to upload.
    # filenames = ["file_1.txt", "file_2.txt"]

    # The directory on your computer that is the root of all of the files in the
    # list of filenames. This string is prepended (with os.path.join()) to each
    # filename to get the full path to the file. Relative paths and absolute
    # paths are both accepted. This string is not included in the name of the
    # uploaded blob; it is only used to find the source files. An empty string
    # means "the current working directory". Note that this parameter allows
    # directory traversal (e.g. "/", "../") and is not intended for unsanitized
    # end user input.
    # source_directory=""

    # The maximum number of processes to use for the operation. The performance
    # impact of this value depends on the use case, but smaller files usually
    # benefit from a higher number of processes. Each additional process occupies
    # some CPU and memory resources until finished. Threads can be used instead
    # of processes by passing `worker_type=transfer_manager.THREAD`.
    # workers=8

    from google.cloud.storage import Client, transfer_manager

    storage_client = Client()
    bucket = storage_client.bucket(bucket_name)

    results = transfer_manager.upload_many_from_filenames(
        bucket, filenames, source_directory=source_directory, max_workers=workers
    )

    for name, result in zip(filenames, results):
        # The results list is either `None` or an exception for each filename in
        # the input list, in order.

        if isinstance(result, Exception):
            print("Failed to upload {} due to exception: {}".format(name, result))
        else:
            print("Uploaded {} to {}.".format(name, bucket.name))

In [44]:
import os
DATA_CACHE_DIR = "/Users/chinmay/Programming/deep_learning/data/edu_fineweb10B"
REMOTE_BUCKET_NAME = "10bt_gpt4"
file_names = os.listdir(DATA_CACHE_DIR)
workers = os.cpu_count() // 2

In [46]:
upload_many_blobs_with_transfer_manager(REMOTE_BUCKET_NAME, file_names, DATA_CACHE_DIR, workers)

Failed to upload edufineweb_train_000008.npy due to exception: A process in the process pool was terminated abruptly while the future was running or pending.
Failed to upload edufineweb_train_000020.npy due to exception: A process in the process pool was terminated abruptly while the future was running or pending.
Failed to upload edufineweb_train_000034.npy due to exception: A process in the process pool was terminated abruptly while the future was running or pending.
Failed to upload edufineweb_val_000000.npy due to exception: A process in the process pool was terminated abruptly while the future was running or pending.
Failed to upload edufineweb_train_000035.npy due to exception: A process in the process pool was terminated abruptly while the future was running or pending.
Failed to upload edufineweb_train_000021.npy due to exception: A process in the process pool was terminated abruptly while the future was running or pending.
Failed to upload edufineweb_train_000009.npy due to ex

# test out if encodings work

In [31]:
import tiktoken
import numpy as np
enc = tiktoken.encoding_for_model("gpt-4") # 'cl100k_base'

In [36]:
tokens = np.load("edu_fineweb10B/edufineweb_val_000000.npy")
print(enc.decode(tokens[:10]))

<|endoftext|>The Independent Jane
For all the love,


In [48]:
len(os.listdir(DATA_CACHE_DIR))

97